### Style Transfer Filter

In [1]:
import os
import gc
import random
import pandas as pd
import pickle
import torch
import json
import math
from collections import defaultdict, Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import logging
import lime
import numpy as np
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, Laplace, Vocabulary
import seaborn as sns
from evaluate import load
import matplotlib.pyplot as plt
import ast
from scipy.stats import pearsonr
import subprocess

random.seed(42)
#import transformers

/gscratch/argon/hjung10/miniconda3/envs/jupyter-notebook/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
def get_memory_usage(idx=""):
    free, available = torch.cuda.mem_get_info()
    print(f"{idx} Free:{free/1000000000:.3f}GB\tAvailable:{available/1000000000:.3f}GB")
#get_memory_usage()

### Loading data

In [4]:
# path to the LLama3-generated synthetic data
SYNTHETIC_DATA_PATH_LLAMA3 = "/gscratch/argon/stelli/reddit_norm/style_transfer/data/output/llama3/"
SAVE_DICTIONARY_PATH = '/gscratch/argon/hjung10/norm_discovery_project/code/synthetic_data_detection/'

# for preprocessing filter
MEDIA_LINK_FITLERED_ID = '/gscratch/argon/stelli/reddit_norm/upvote_prediction/processed_upvotes_media_edit/'
PREPROCESSING_FILTER_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/preprocessing_filter/passed_filter/'   # passed_filter -> data that passed filter and is going to the next filters
PREPROCESSING_FILTERED_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/preprocessing_filter/filtered/'  # filtered -> data that has been filtered

# Lexical filter
LINGUISTIC_FILTER_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/lexical_filter/passed_filter/'
LINGUISTIC_FILTERED_DIR= '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/lexical_filter/filtered/'

# fluency filter
PERPLEXITY_FORMATTED_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/fluency/dialoGPT_formatted/'  # stores all intermediate perplexity computations so we don't need to repeat compute again in the future
FLUENCY_FILTER_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/fluency_filter/passed_filter/'
FLUENCY_FILTERED_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/fluency_filter/filtered/'

# content preservation filter
CONTENT_PRES_FILTER_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/content_preservation_filter/passed_filter/'
CONTENT_PRES_FILTERED_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/filters/content_preservation_filter/filtered/'
BERTSCORE_FORMATTED_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/content_preservation/bertscore-generated-formatted/' # stores all intermediate bertscore computations so we don't need to repeat compute again in the future

# SLURM scripts directory (see valueScope/style_transfer/filter/scripts/ directory)
SCRIPTS_DIR = '/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/scripts/'

directories_llama3 = os.listdir(SYNTHETIC_DATA_PATH_LLAMA3)
directories_llama3.remove('dedup.py')
directories_llama3 = [jsonl for jsonl in directories_llama3 if "_old" not in jsonl]

# subreddits and norm dimension
subreddits = ['askmen', 'askwomen', 'asktransgender', 'askscience', 'shittyaskscience', 'asksciencediscussion', 'democrats', 'republican',
             'libertarian', 'stocks', 'pennystocks', 'wallstreetbets', 'wallstreetbetsnew']
norm_dimensions = ['formality', 'humor', 'sarcasm', 'politeness', 'supportiveness']

In [5]:
filter_phrases = ['I apologize, but','not able to fulfill this request','cannot fulfill your request', "I cannot provide a rewritten"]
split_phrases = ['"\n\nOriginal Comment', '"\n\nThis', '"\n\nRating', '\n\nRating', '"\n\nPlease', '"\n\nNote', '"\n\nRewritten', '"\n\nOriginal', '"\n\nRemember', '"\n\nI rewrote', '"\n\nI hope', '"\n\nI rated', '"\n\nI Changed', '"\n\nI kept', '"\n\nI tried', '"\n\nThe original', '"\n\nI\'ve']
split_phrases_after = ['\n\nMy answer: "']
def filter_comment(comment):
    # filter out abstains
    if any([phrase in comment for phrase in filter_phrases]):
        return None

    # to filter out instances where the model just generates "NOOOOOOOOOOOO..."
    if "NOOOOOOOOOOOOOOOOOOOOOOOO" in comment: 
        return None

    # filter out unnecessary phrasess
    for split_phrase in split_phrases:
        if split_phrase in comment:
            return comment.split(split_phrase)[0]
    if "Here\'s the rewritten" in comment or 'Here is the rewritten' in comment or "Here\'s a" in comment or 'Here is a':
        if "\n\n" in comment:
            index = comment.find("\n\n")
            return comment[index+2:]
        if "\n" in comment:
            index = comment.find("\n")
            return comment[index:]
    for split_phrase in split_phrases_after:
        if split_phrase in comment:
            return comment.split(split_phrase)[1]
    return comment

# simple string preprocessing; standardizes quotations to use "" as the model
#can add various single/double quotes in their response
def standardize_quotations(comment):
    if comment == None:
        return comment

    comment_cleaned = ""
    if comment[0] == '"' and comment[len(comment) - 1] == '"':
        comment_cleaned = comment[1:len(comment)-1]
    elif comment[0] == "'" and comment[len(comment) - 1] == "'":
        comment_cleaned = comment[1:len(comment)-1]
    else:
        comment_cleaned = comment

    comment_single_quotation = ''
    for char in comment_cleaned:
        comment_single_quotation += char

    return comment_single_quotation

# preprocess each style transfer generations
def process_data(df):
    _data = []
    for _, row in df.iterrows():
        original_id = row['id']
        _data.append({'id': original_id+"-0", 'id-original': original_id,'comment': row['original_comment'], 'rating': row['original_rating'], 'post_title': row['submission_title']})
        for scale in ['1','2','3','4','5']:
            filtered_comment = filter_comment(row[scale])
            if filtered_comment is not None:
                _data.append({'id': original_id+"-"+scale,'id-original':original_id, 'comment': filtered_comment, 'rating': int(scale), 'post_title': row['submission_title']})
    return _data

In [9]:
# reads in synthetic data from the directories
def select_synthetic_data(PATH, directories):
    reddit_to_dimension_comments = dict()
    reddit_to_longest = dict()
    
    for jsonl_file in directories:   

        num_duplicates = 0
        set_added = set()
        # load original comments data
        with open(os.path.join(PATH, jsonl_file), "r") as f:
            comments = []
            for line in f.readlines():
                json_line = json.loads(line)
                id = json_line['id']

                # preventing any potential duplicates in our data
                if id in set_added:
                    num_duplicates += 1
                    continue
                set_added.add(id)
                comments.append(json_line)

            # select the first 50K comments
            reddit_to_dimension_comments[jsonl_file] = comments[:50000]
            print(jsonl_file)
            print(num_duplicates)
            print(len(reddit_to_dimension_comments[jsonl_file]))
            print("------")

    return reddit_to_dimension_comments#, reddit_to_longest

In [10]:
# reading in all the style transfer data
reddit_to_dimension_comments = select_synthetic_data(SYNTHETIC_DATA_PATH_LLAMA3, directories_llama3)

askmen_formality.jsonl
0
50000
------
libertarian_politeness.jsonl
0
50000
------
askscience_humor.jsonl
0
50000
------
askwomen_length.jsonl
0
50000
------
askwomen_supportiveness.jsonl
0
50000
------
shittyaskscience_sarcasm.jsonl
0
50000
------
libertarian_supportiveness.jsonl
0
50000
------
askscience_length.jsonl
0
50000
------
asksciencediscussion_length.jsonl
0
50000
------
democrats_sarcasm.jsonl
0
50000
------
asktransgender_length.jsonl
0
50000
------
asksciencediscussion_politeness.jsonl
0
50000
------
stocks_humor.jsonl
0
50000
------
republican_politeness.jsonl
0
50000
------
libertarian_length.jsonl
0
50000
------
asktransgender_formality.jsonl
0
50000
------
shittyaskscience_humor.jsonl
0
50000
------
wallstreetbets_humor.jsonl
0
50000
------
stocks_sarcasm.jsonl
0
50000
------
askmen_humor.jsonl
0
50000
------
pennystocks_politeness.jsonl
0
50000
------
democrats_supportiveness.jsonl
0
50000
------
wallstreetbetsnew_politeness.jsonl
0
50000
------
stocks_supportiveness.

### Preprocessing Filter
- Filtering out edited comments, comments with only links, media/video, recent posts that may not have gotten enough time to garner community attention and upvotes, empty strings, and synthetic data that did not style transfer (e.g. same as original)
- PREPROCESSING_FILTERED_DIR and PREPROCESSING_FILTER_DIR contains jsonl files. Both can be read as dictionaries (key being the ID, value being the first 20 characters). PREPROCESSING_FILTER_DIR represents the dictionary that passed the preprocessed filter, while the PREPROCESSING_FILTERED_DIR represents ones that were filtered out 

In [11]:
FILTERED_DIR = '/gscratch/argon/stelli/reddit_norm/upvote_prediction/data/processed_upvotes_filter/'

# this function gathers all the filtered IDs
def gather_filtered_out_ids(FILTERED_DIR):
    filtered_ids = set()
    for file in os.listdir(FILTERED_DIR):
        if '.json' in file:
            with open(FILTERED_DIR + file, 'rb') as f:
                data = json.load(f)

            set_ = set()
            for item in data['all_filters']:

                # standardize the reddit comment IDs
                if "t1_" in item:
                    set_.add(item[3:])
                else:
                    set_.add(item)
            filtered_ids.update(set_)
    return filtered_ids

filtered_ids = gather_filtered_out_ids(FILTERED_DIR)

In [15]:
# this function gathers existing IDs in a set that passed the filter so we don't need to append
# those IDs again in our files within PATH. This 
def read_existing_dictionary(PATH):
    reddit_to_data = dict()
    for file in os.listdir(PATH):
        if ".jsonl" in file:
            set_data = set()
            with open(PATH + file, 'r') as f:
                for line in f:
                    line = json.loads(line)
                    set_data.update(line.keys())   # adding the keys (e.g. comment IDs)

            reddit_to_data[file] = set_data
    return reddit_to_data

# this function gathers existing IDs as well as their mapped values in a dictionary that passed the filter
def read_existing_dictionary_key_value(PATH):
    reddit_to_data = dict()
    for file in os.listdir(PATH):
        if ".jsonl" in file:
            data_dict = dict()
            with open(PATH + file, 'r') as f:
                for line in f:
                    line = json.loads(line)
                    data_dict.update(line)     # adding the ID to comments together

            reddit_to_data[file] = data_dict
    return reddit_to_data

In [16]:
# PATH and file_name of the file to append to for list_dict
# previous_reddit_to_data represents the existing data so we don't append data that's already in the file
def save_jsonl(PATH, file_name, list_dict, previous_reddit_to_data):
    new_added = 0
    with open(PATH + file_name, 'a') as f:
        for dict_ in list_dict:
            if file_name not in previous_reddit_to_data or (file_name in previous_reddit_to_data and list(dict_.keys())[0] not in previous_reddit_to_data[file_name]):
                new_added += 1
                json.dump(dict_, f)
                f.write('\n')
    return new_added

In [14]:
# parsing through all the synthetic comments, keeping track of which comments were filtered or not,
# and updating our files accordingly
def apply_preprocessed_filter():
    total_considered = 0
    total_filtered = 0
    total_left = 0
    reddit_to_preprocesed_data = read_existing_dictionary(PREPROCESSING_FILTER_DIR)
    reddit_filtered_data = read_existing_dictionary(PREPROCESSING_FILTERED_DIR)
    
    # parsing through the style transferred comments
    for reddit_json, list_synthetic in reddit_to_dimension_comments.items():
        reddit = reddit_json.split('_')[0]
        dimension = reddit_json.split('_')[1].split('.json')[0]
        print(reddit)
        print(dimension)
        print("Number of IDs before preprocessed filter: " + str(len(list_synthetic)))
        total_considered += len(list_synthetic)
    
        # parsing through the comments, checking against the preprocessed filter
        list_dict = []
        list_filtered = []
        for comment in list_synthetic:
            original_id = comment['id']

            # standardize the ID format
            if "t1_" in original_id:
                original_id = original_id[3:]
            original_comment = comment['original_comment']
    
            # filter out edited comments, comments with only links, media/video, and recent posts that 
            # may not have gotten enough time to garner community attention and upvotes
            id_to_comment = dict()
            if original_id not in filtered_ids:
                id_to_comment[original_id] = original_comment[:20]
                list_dict.append(id_to_comment)
            else:
                id_to_comment[original_id] = original_comment[:20]
                list_filtered.append(id_to_comment)
        print("Number of IDs that passed the preprocessed filter: " + str(len(list_dict)))
        if len(list_dict) < 10000:
            print("Needs more!!")
        total_left += len(list_dict)
        total_filtered += len(list_filtered)
    
        # saving to the file
        file_name =reddit + '_' + dimension + '.jsonl'
        added = save_jsonl(PREPROCESSING_FILTER_DIR, file_name, list_dict, reddit_to_preprocesed_data)

        # saving filtered data
        file_name =reddit + '_' + dimension + '.jsonl'
        added_filtered = save_jsonl(PREPROCESSING_FILTERED_DIR, file_name, list_filtered, reddit_filtered_data)
    
        print("Number of new IDs that passed filter added: " + str(added))
        print("Number of new IDs that were filtered: " + str(added_filtered))
        print("-------------------")
    
    print("Total number of original IDs considered: " + str(total_considered))
    print("Total number of original IDs filtered: " + str(total_filtered))
    print("% of the original IDs filtered: " + str((total_filtered) / total_considered))
    print("Total number of original IDs left: " + str(total_left))
    print()
    print("Total number of synthetic comments considered: " + str(total_considered * 5))
    print("Total number of synthetic comments filtered: " + str(total_filtered * 5))
    print("Total number of synthetic comments left: " + str(total_left * 5))

In [18]:
apply_preprocessed_filter()

askmen
formality
Number of IDs before preprocessed filter: 50000
Number of IDs that passed the preprocessed filter: 49154
Number of new IDs that passed filter added: 49154
Number of new IDs that were filtered: 846
-------------------
libertarian
politeness
Number of IDs before preprocessed filter: 50000
Number of IDs that passed the preprocessed filter: 42473
Number of new IDs that passed filter added: 42473
Number of new IDs that were filtered: 7527
-------------------
askscience
humor
Number of IDs before preprocessed filter: 50000
Number of IDs that passed the preprocessed filter: 45293
Number of new IDs that passed filter added: 45293
Number of new IDs that were filtered: 4707
-------------------
askwomen
length
Number of IDs before preprocessed filter: 50000
Number of IDs that passed the preprocessed filter: 48847
Number of new IDs that passed filter added: 48847
Number of new IDs that were filtered: 1153
-------------------
askwomen
supportiveness
Number of IDs before preprocesse

### Lexical Filter
- Filtering out lexical words ("here's the rewritten comments..."), abstains, no style transfer (e.g. synthetic comment being the same as the original), empty strings
- Given that we are lexically preprocessing the strings, use the LINGUISTIC_FILTER_DIR from this point

In [19]:
def apply_lexical_filter():
    total_synthetic_considered = 0
    total_synthetic_filtered = 0
    total_synthetic_left = 0
    reddit_to_preprocesed_data = read_existing_dictionary(PREPROCESSING_FILTER_DIR)
    reddit_to_lexical_data = read_existing_dictionary(LINGUISTIC_FILTER_DIR)
    reddit_to_lexical_filtered = read_existing_dictionary(LINGUISTIC_FILTERED_DIR)

    # sanity check
    total_added = 0
    total_filtered = 0

    # similar to the preprocessing filter implementation, but we decided to implement these filters
    # separately to know how much data is being filtered at each step
    for reddit_json, list_synthetic in reddit_to_dimension_comments.items():
        reddit = reddit_json.split('_')[0]
        dimension = reddit_json.split('_')[1].split('.json')[0]
        print(reddit)
        print(dimension)
    
        list_dict = []
        list_filtered = []
        # parsing through each set of comments (e.g. 1 original, 5 synthetic); 
        for comment in list_synthetic:
            original_id = comment['id']
            original_comment = comment['original_comment']
            submission_title = comment['submission_title']

            # standardize the ID format
            if "t1_" in original_id:
                original_id = original_id[3:]
    
            # original implementation combined both lexical and preprocessed filter together since the
            # code is essentially the same, but we decided it would be better to separate the two filters out
            if original_id not in reddit_to_preprocesed_data[reddit_json]:
                continue
    
            # parsing through each style transfer scale
            for scale_idx in range(1, 6):
                synthetic_id = original_id + '-' + str(scale_idx)
                synthetic_comment = comment[str(scale_idx)]
                total_synthetic_considered += 1
    
                # filtering lexically (e.g. abstains, additional strings)
                filtered_comment = filter_comment(synthetic_comment)
    
                # checking if abstains or other filtered strings, no style transfer being done, empty strings, single character generation
                id_to_comment = dict()
                if filtered_comment == None or original_comment == synthetic_comment or filtered_comment.strip() == "":
                    id_to_comment[synthetic_id] = filtered_comment
                    list_filtered.append(id_to_comment)     # add to list of filtered data
                    total_synthetic_filtered += 1
                else:
                     # standardizing quotations to clean the data a bit better
                    filtered_comment = standardize_quotations(filtered_comment)

                    # edge case where there's only one character generation; filtering that out too
                    if len(filtered_comment) <= 1:
                        id_to_comment[synthetic_id] = filtered_comment
                        list_filtered.append(id_to_comment)     # add to list of filtered data
                        total_synthetic_filtered += 1
                    else:
                        # NOTE: We need to add the original comment for easier bertscore computation instead of going back into
                        # our data and fetching them
                        id_to_comment[synthetic_id] = (filtered_comment, original_comment)
                        list_dict.append(id_to_comment)     # add to list of data that passed the filter
                        total_synthetic_left += 1
    
        # saving to the file
        file_name =reddit + '_' + dimension + '.jsonl'
        added = save_jsonl(LINGUISTIC_FILTER_DIR, file_name, list_dict, reddit_to_lexical_data)
    
        # saving filtered data
        file_name =reddit + '_' + dimension + '.jsonl'
        added_filtered = save_jsonl(LINGUISTIC_FILTERED_DIR, file_name, list_filtered, reddit_to_lexical_filtered)
    
        print("Number of new IDs that passed filter added: " + str(added))
        print("Number of new IDs that were filtered: " + str(added_filtered))
        print("-------------------")

        total_added += added
        total_filtered += added_filtered
    
    
    print("Total number of synthetic comments considered: " + str(total_synthetic_considered))
    print("Total number of synthetic comments filtered: " + str(total_synthetic_filtered))
    print("% of the synthetic comments filtered: " + str((total_synthetic_filtered) / total_synthetic_considered))
    print("Total number of synthetic comments left: " + str(total_synthetic_left))

    print("total added: " + str(total_added))
    print("total filtered: " + str(total_filtered))

In [20]:
apply_lexical_filter()

askmen
formality
Number of new IDs that passed filter added: 245597
Number of new IDs that were filtered: 173
-------------------
libertarian
politeness
Number of new IDs that passed filter added: 212307
Number of new IDs that were filtered: 58
-------------------
askscience
humor
Number of new IDs that passed filter added: 226435
Number of new IDs that were filtered: 30
-------------------
askwomen
length
Number of new IDs that passed filter added: 243024
Number of new IDs that were filtered: 1211
-------------------
askwomen
supportiveness
Number of new IDs that passed filter added: 244210
Number of new IDs that were filtered: 25
-------------------
shittyaskscience
sarcasm
Number of new IDs that passed filter added: 197662
Number of new IDs that were filtered: 53
-------------------
libertarian
supportiveness
Number of new IDs that passed filter added: 212214
Number of new IDs that were filtered: 61
-------------------
askscience
length
Number of new IDs that passed filter added: 22

### Fluency Filter (PPL)
- Employing DialoGPT, a model finetuned on 140M Reddit conversations, to compute perplexity of synthetic comments and filter them accordingly
- TO ensure that the synthetic comments are as fluent as the original, human-written ones, we exclude synthetic comments with perplexity values outside the range of +-1 standard deviation (6860 PPL) from the mean perplexity (2747 PPL) of the original comments. We filter everything else.
- At this stage, we employed SLURM and sent GPU jobs to compute the perplexity of the original and synthetic comments.
- The GPU sbatch script will call on 'perplexity_compute.py' which is provided within valueScope/style_transfer/filter/scripts

In [21]:
# reading in prior style transfer generations to avoid computing the perplexity for new generations
perplexity_computed = read_existing_dictionary(PERPLEXITY_FORMATTED_DIR)
reddit_to_lexical_data = read_existing_dictionary_key_value(LINGUISTIC_FILTER_DIR)

In [22]:
# to automate bash scripting
def write_script(subreddit, dimension):
    script = """#!/bin/bash
#SBATCH --job-name=perplexity_sbatch_{SUBREDDIT}_{DIMENSION}
#SBATCH --mail-type=FAIL,INVALID_DEPEND
#SBATCH --mail-user=
#SBATCH --account=
#SBATCH --partition=ckpt
#SBATCH --nodes=1
#SBATCH --cpus-per-task=5
#SBATCH --mem=100G
#SBATCH --gpus=1
#SBATCH --time=2-00:00:00 # Max runt:ime in DD-HH:MM:SS format.
#SBATCH --chdir=/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/
#SBATCH --export=all
#SBATCH --output=/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/logs/%x-%j.out # where STDOUT goes
#SBATCH --error=/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/logs/%x-%j.err # where STDERR goes

# Your programs to run.
source /mmfs1/home/hjung10/.bashrc
conda activate jupyter-notebook
cd /gscratch/argon/hjung10

CUDA_LAUNCH_BLOCKING=1 python /gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/scripts/perplexity_compute.py --subreddit '{SUBREDDIT}' --dimension '{DIMENSION}'"""

    script = script.replace('{SUBREDDIT}', subreddit)
    script = script.replace('{DIMENSION}', dimension)
    return script

In [23]:
# first step is to compute perplexity values for any new style transfer generations
def determine_new_style_transfers_needed(reddit_to_lexical_data, perplexity_computed):
    reddit_to_compute_needed = dict()
    
    for reddit_json, dict_synthetic in reddit_to_lexical_data.items():
        # contains set of synthetic comment IDs whose perplexity have already been computed
        comments_computed_set = None
        if reddit_json in perplexity_computed:
            comments_computed_set = perplexity_computed[reddit_json]
    
        # dictionary to keep track of comments that we need to compute perplexity for
        dict_compute = dict() 
        int_computed = 0
        for synthetic_id, tuple_comments in dict_synthetic.items():
    
            # the perplexity for the synthetic comment has not been computed; add to compute list
            if comments_computed_set == None or synthetic_id not in comments_computed_set:
                dict_compute[synthetic_id] = tuple_comments
            else:
                int_computed += 1
        print(reddit_json)
        print("Number of synthetic comments needed for compute: " + str(len(dict_compute)))
        print("Number of synthetic comments already computed: " + str(int_computed))
        reddit_to_compute_needed [reddit_json] = dict_compute
        print()
    return reddit_to_compute_needed

In [1]:
def compute_perplexity(reddit_to_compute_needed):
    for file_name, content in reddit_to_compute_needed.items():
        # only executing batch script if new perplexity needs to be computed
        if len(content) > 0:
            reddit = file_name.split('_')[0]
            dimension = file_name.split('_')[1].split('.')[0]
            
            script = write_script(reddit, dimension)
            sbatch_file = 'run_perplexity.sbatch'
            with open(SCRIPTS_DIR + sbatch_file, 'w') as file:
                file.write(script)
    
            print(reddit)
            print(dimension)
            execute_string = SCRIPTS_DIR + sbatch_file
            subprocess.run(['sbatch', execute_string])
            print("----")

In [25]:
# determining which subreddit-dimension has new style transferred comments that needs to perplexity computation
reddit_to_compute_needed = determine_new_style_transfers_needed(reddit_to_lexical_data, perplexity_computed)
compute_perplexity(reddit_to_compute_needed)

askmen_formality.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 245597

libertarian_politeness.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 212307

askscience_humor.jsonl
Number of synthetic comments needed for compute: 28689
Number of synthetic comments already computed: 197746

askwomen_length.jsonl
Number of synthetic comments needed for compute: 35428
Number of synthetic comments already computed: 207596

askwomen_supportiveness.jsonl
Number of synthetic comments needed for compute: 34730
Number of synthetic comments already computed: 209480

shittyaskscience_sarcasm.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 197662

libertarian_supportiveness.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 212214

askscience_length.jsonl
Number of synthetic comments ne

In [26]:
def apply_fluency_filter():
    # rereading in new perplexity computations so we do not add duplicate comments in the file
    reddit_to_ppl_data = read_existing_dictionary(FLUENCY_FILTER_DIR)
    reddit_to_ppl_filtered = read_existing_dictionary(FLUENCY_FILTERED_DIR)
    
    # containing the perplexity numbers so we can easily check
    perplexity_computed = read_existing_dictionary_key_value(PERPLEXITY_FORMATTED_DIR)
    
    # data from the prior filter (continuing where we left off)
    reddit_to_lexical_data = read_existing_dictionary_key_value(LINGUISTIC_FILTER_DIR)

    # sanity check
    total_added = 0
    total_filtered = 0
    
    total_synthetic_considered = 0
    total_synthetic_filtered = 0
    total_synthetic_left = 0
    for reddit_json, dict_synthetic in reddit_to_lexical_data.items():
        print(reddit_json)
    
        list_dict = []
        list_filtered = []
        for synthetic_id, list_comments in dict_synthetic.items():
            total_synthetic_considered += 1
    
            if reddit_json in perplexity_computed:
                perplexity = 0
                if synthetic_id in perplexity_computed[reddit_json]:
                    perplexity = perplexity_computed[reddit_json][synthetic_id]
                else:
                    print(list_comments)
    
                # check perplexity within the threshold
                id_to_comments = dict()

                if perplexity >= (2747-6860) and perplexity <= (2747+6860):
                    total_synthetic_left +=1
                    id_to_comments[synthetic_id] = list_comments
                    list_dict.append(id_to_comments)
                else:
                    total_synthetic_filtered += 1
                    id_to_comments[synthetic_id] = list_comments
                    list_filtered.append(id_to_comments)
         # saving to the file
        added = save_jsonl(FLUENCY_FILTER_DIR, reddit_json, list_dict, reddit_to_ppl_data)
    
        # saving filtered data
        added_filtered = save_jsonl(FLUENCY_FILTERED_DIR, reddit_json, list_filtered, reddit_to_ppl_filtered)
        print("Number of new IDs that passed filter added: " + str(added))
        print("Number of new IDs that were filtered: " + str(added_filtered))
        print("-------------------")
        
        total_added += added
        total_filtered += added_filtered
    
    print("Total number of synthetic comments considered: " + str(total_synthetic_considered))
    print("Total number of synthetic comments filtered: " + str(total_synthetic_filtered))
    print("% of the synthetic comments filtered: " + str((total_synthetic_filtered) / total_synthetic_considered))
    print("Total number of synthetic comments left: " + str(total_synthetic_left))

    
    print("total added: " + str(total_added))
    print("total filtered: " + str(total_filtered))

In [27]:
# only execute this cell once the perplexity scores have been all computed (e.g. GPU jobs are done)
apply_fluency_filter()

askmen_formality.jsonl
Number of new IDs that passed filter added: 244405
Number of new IDs that were filtered: 1192
-------------------
libertarian_politeness.jsonl
Number of new IDs that passed filter added: 212247
Number of new IDs that were filtered: 60
-------------------
askscience_humor.jsonl
Number of new IDs that passed filter added: 226335
Number of new IDs that were filtered: 100
-------------------
askwomen_length.jsonl
Number of new IDs that passed filter added: 230621
Number of new IDs that were filtered: 12403
-------------------
askwomen_supportiveness.jsonl
Number of new IDs that passed filter added: 244065
Number of new IDs that were filtered: 145
-------------------
shittyaskscience_sarcasm.jsonl
Number of new IDs that passed filter added: 197217
Number of new IDs that were filtered: 445
-------------------
libertarian_supportiveness.jsonl
Number of new IDs that passed filter added: 212107
Number of new IDs that were filtered: 107
-------------------
askscience_lengt

### Content Preservation Filter
- Here, we employ BERTScore, which has been shown to align with human judgments in prior works. After qualitative investigation of various BERTScore values and the content preservations between original vs. synthetic comments, we decided to use BERTScore=0.5 as a threshold, removing synthetic comments whose BERTScore values fall below 0.5. 
- BERTScore computed using the model "microsoft/deberta-xlarge-mnli" which best aligns with human judgement according to the authors (https://github.com/Tiiiger/bert_score#readme)
- @inproceedings{bert-score,
  title={BERTScore: Evaluating Text Generation with BERT},
  author={Tianyi Zhang* and Varsha Kishore* and Felix Wu* and Kilian Q. Weinberger and Yoav Artzi},
  booktitle={International Conference on Learning Representations},
  year={2020},
  url={https://openreview.net/forum?id=SkeHuCVFDr}}
- At this stage, we also use SLURM and send GPU jobs to compute the bert scores. The GPU sbatch script will call on 'bert-score-compute.py' which is provided within valueScope/style_transfer/filter/scripts

In [36]:
# reading in prior style transfer generations to avoid computing the perplexity for new generations
bertscore_computed = read_existing_dictionary(BERTSCORE_FORMATTED_DIR)
reddit_to_ppl_data = read_existing_dictionary_key_value(FLUENCY_FILTER_DIR)

In [37]:
# to automate bash scripting
def write_script_bertscore(subreddit, dimension):
    script = """#!/bin/bash
#SBATCH --job-name=bertscore_sbatch_{SUBREDDIT}_{DIMENSION}
#SBATCH --mail-type=FAIL,INVALID_DEPEND
#SBATCH --mail-user=
#SBATCH --account=
#SBATCH --partition=ckpt
#SBATCH --nodes=1
#SBATCH --cpus-per-task=5
#SBATCH --mem=100G
#SBATCH --gpus=1
#SBATCH --time=2-00:00:00 # Max runt:ime in DD-HH:MM:SS format.
#SBATCH --chdir=/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/
#SBATCH --export=all
#SBATCH --output=/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/logs/%x-%j.out # where STDOUT goes
#SBATCH --error=/gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/logs/%x-%j.err # where STDERR goes

# Your programs to run.
source /mmfs1/home/hjung10/.bashrc
export HF_HOME=/gscratch/scrubbed/hjung10/{SUBREDDIT}-{DIMENSION}
conda activate jupyter-notebook
cd /gscratch/argon/hjung10

CUDA_LAUNCH_BLOCKING=1 python /gscratch/argon/hjung10/norm_discovery_project/code/style_transfer_filter/scripts/bert-score-compute.py --subreddit '{SUBREDDIT}' --dimension '{DIMENSION}'"""

    script = script.replace('{SUBREDDIT}', subreddit)
    script = script.replace('{DIMENSION}', dimension)
    return script

In [38]:
# determine new bertscore computations needed
def determine_new_bertscores_needed(reddit_to_ppl_data, bertscore_computed):
    reddit_to_compute_needed = dict()

    for reddit_json, dict_synthetic in reddit_to_ppl_data.items():
        # contains set of synthetic comment IDs whose perplexity have already been computed
        comments_computed_set = None
        if reddit_json in bertscore_computed:
            comments_computed_set = bertscore_computed[reddit_json]

        # dictionary to keep track of comments that we need to compute perplexity for
        dict_compute = dict() 
        int_computed = 0
        for synthetic_id, tuple_comments in dict_synthetic.items():
            if comments_computed_set == None or synthetic_id not in comments_computed_set:
                dict_compute[synthetic_id] = tuple_comments
            else:
                int_computed += 1
        print(reddit_json)
        print("Number of synthetic comments needed for compute: " + str(len(dict_compute)))
        print("Number of synthetic comments already computed: " + str(int_computed))
        reddit_to_compute_needed [reddit_json] = dict_compute
        print()
    return reddit_to_compute_needed

In [39]:
def compute_bertscore(reddit_to_compute_needed):
    for file_name, content in reddit_to_compute_needed.items():
        # only executing batch script if new perplexity eneds to be computed
        if len(content) > 0:
            reddit = file_name.split('_')[0]
            dimension = file_name.split('_')[1].split('.')[0]
            
            script = write_script_bertscore(reddit, dimension)
            sbatch_file = 'run_bertscore.sbatch'
            with open(SCRIPTS_DIR + sbatch_file, 'w') as file:
                file.write(script)
    
            print(reddit)
            print(dimension)
            execute_string = SCRIPTS_DIR + sbatch_file
            subprocess.run(['sbatch', execute_string])
            print("----")

In [40]:
# determining which subreddit-dimension has new style transferred comments that needs to perplexity computation
reddit_to_bertscore_compute_needed = determine_new_bertscores_needed(reddit_to_ppl_data, bertscore_computed)
compute_bertscore(reddit_to_bertscore_compute_needed)

askmen_formality.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 244405

libertarian_politeness.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 212247

askscience_humor.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 226335

askwomen_length.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 230621

askwomen_supportiveness.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 244065

shittyaskscience_sarcasm.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 197217

libertarian_supportiveness.jsonl
Number of synthetic comments needed for compute: 0
Number of synthetic comments already computed: 212107

askscience_length.jsonl
Number of synthetic comments needed for com

In [41]:
def apply_content_preservation_filter():
    # rereading in new perplexity computations so we do not add duplicate comments in the file
    reddit_to_bertscore_data = read_existing_dictionary(CONTENT_PRES_FILTER_DIR)
    reddit_to_bertscore_filtered = read_existing_dictionary(CONTENT_PRES_FILTERED_DIR)
    
    # containing the perplexity numbers so we can easily check
    bertscore_computed = read_existing_dictionary_key_value(BERTSCORE_FORMATTED_DIR)
    
    # data from the prior filter (continuing where we left off)
    reddit_to_ppl_filter_data = read_existing_dictionary_key_value(FLUENCY_FILTER_DIR)

    # sanity check
    total_added = 0
    total_filtered = 0
    

    total_synthetic_considered = 0
    total_synthetic_filtered = 0
    total_synthetic_left = 0
    for reddit_json, dict_synthetic in reddit_to_ppl_filter_data.items():
        print(reddit_json)
    
        list_dict = []
        list_filtered = []
        for synthetic_id, list_comments in dict_synthetic.items():
            total_synthetic_considered += 1
    
            if reddit_json in bertscore_computed:
                bertscore = 0
                if synthetic_id in bertscore_computed[reddit_json]:
                    bertscore = bertscore_computed[reddit_json][synthetic_id]['f1'][0]
                #else:
                    #print(list_comments)
    
                # check perplexity within the threshold
                id_to_comments = dict()
                if bertscore >= 0.5:
                    total_synthetic_left += 1
                    id_to_comments[synthetic_id] = list_comments
                    list_dict.append(id_to_comments)
                else:
                    total_synthetic_filtered += 1
                    id_to_comments[synthetic_id] = list_comments
                    list_filtered.append(id_to_comments)
        # saving to the file
        added = save_jsonl(CONTENT_PRES_FILTER_DIR, reddit_json, list_dict, reddit_to_bertscore_data)
    
        # saving filtered data
        added_filtered = save_jsonl(CONTENT_PRES_FILTERED_DIR, reddit_json, list_filtered, reddit_to_bertscore_filtered)
        print("Number of new IDs that passed filter added: " + str(added))
        print("Number of new IDs that were filtered: " + str(added_filtered))
        print("-------------------")

        total_added += added
        total_filtered += added_filtered
    
    print("Total number of synthetic comments considered: " + str(total_synthetic_considered))
    print("Total number of synthetic comments filtered: " + str(total_synthetic_filtered))
    print("% of the synthetic comments filtered: " + str((total_synthetic_filtered) / total_synthetic_considered))
    print("Total number of synthetic comments left: " + str(total_synthetic_left))

    print("total added: " + str(total_added))
    print("total filtered: " + str(total_filtered))

In [42]:
# only execute cell once all bertscore has been computed (e.g. GPU jobs are done)
apply_content_preservation_filter()

askmen_formality.jsonl
Number of new IDs that passed filter added: 199845
Number of new IDs that were filtered: 44560
-------------------
libertarian_politeness.jsonl
Number of new IDs that passed filter added: 171722
Number of new IDs that were filtered: 40525
-------------------
askscience_humor.jsonl
Number of new IDs that passed filter added: 210169
Number of new IDs that were filtered: 16166
-------------------
askwomen_length.jsonl
Number of new IDs that passed filter added: 182313
Number of new IDs that were filtered: 48308
-------------------
askwomen_supportiveness.jsonl
Number of new IDs that passed filter added: 195901
Number of new IDs that were filtered: 48164
-------------------
shittyaskscience_sarcasm.jsonl
Number of new IDs that passed filter added: 129351
Number of new IDs that were filtered: 67866
-------------------
libertarian_supportiveness.jsonl
Number of new IDs that passed filter added: 162696
Number of new IDs that were filtered: 49411
-------------------
asks